# Libraries and Mock Data

## Libraries

In [1]:
# =======================
# Standard & System Libraries
# =======================
import json
import os
import pickle
import random
import re
import string

# =======================
# Numerical and Data Libraries
# =======================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

# =======================
# NLP Libraries
# =======================
import nltk
import spacy
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from sentence_transformers import SentenceTransformer
from textblob import TextBlob

# =======================
# Machine Learning & Deep Learning
# =======================
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, Subset
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import (
    CountVectorizer,
    TfidfVectorizer,
    ENGLISH_STOP_WORDS
)
from sklearn.manifold import TSNE
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split

# =======================
# Transformers & Hugging Face
# =======================
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    BertForSequenceClassification,
    BertTokenizer,
    DataCollatorWithPadding,
    EarlyStoppingCallback,
    Trainer,
    TrainingArguments
)

# =======================
# Topic Modeling & Clustering
# =======================
import hdbscan
from bertopic import BERTopic
from bertopic.representation import (
    KeyBERTInspired,
    LlamaCPP,
    MaximalMarginalRelevance,
    PartOfSpeech
)

# =======================
# Gensim (Topic Coherence)
# =======================
#from gensim.models import CoherenceModel
#from gensim.corpora import Dictionary

# =======================
# Visualization
# =======================
import datamapplot
import umap

# =======================
# Hugging Face Datasets
# =======================
from datasets import Dataset, concatenate_datasets, load_dataset

# =======================
# Other Libraries
# =======================
from llama_cpp import Llama

# =======================
# Initial Setup
# =======================
nltk.download('wordnet')
nlp = spacy.load("en_core_web_sm")

# =======================
# Environment Variables
# =======================
TF_ENABLE_ONEDNN_OPTS = 0


/home/jordi/.local/lib/python3.10/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(
2025-04-24 16:34:54.410276: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-24 16:34:54.440532: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-24 16:34:55.291343: W tensorflow/compiler/tf2tensorrt/uti

## Mock Data
The following mock data has been generated with ChatGPT.

Mock Data Labeled Datasets HS

In [4]:
def generate_mock_hate_labeled_data():
    texts = [
        "This is a terrible comment.",  # hate
        "You're the worst person ever.",  # hate
        "I can't stand your opinions.",  # hate
        "This is hateful and unacceptable.",  # hate
        "I will report this offensive post.",  # hate
        "Stop posting such hateful things.",  # hate
        "This content is deeply offensive.",  # hate
        "This is pure hate speech.",  # hate
        "You should be banned for this.",  # hate
        "This comment is abusive.",  # hate
        "Great job on the project!",  # non-hate
        "I agree with this viewpoint.",  # non-hate
        "This is a very informative post.",  # non-hate
        "Well done on explaining this topic.",  # non-hate
        "I support this initiative fully.",  # non-hate
        "This is a good contribution to the discussion.",  # non-hate
        "Thank you for sharing this.",  # non-hate
        "This is a positive and helpful comment.",  # non-hate
        "Excellent work, keep it up!",  # non-hate
        "I appreciate your insight on this matter."  # non-hate
    ]
    labels = [1] * 10 + [0] * 10  # First 10 are hate (1), last 10 are non-hate (0)
    return pd.DataFrame({'text': texts, 'label': labels})

mock_data_hate_labeled = generate_mock_hate_labeled_data()

Mock Data TikTok Comments

In [5]:
mock_data_comments = {
    "cid": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
    "text": [
        # Non-hate comments
        "I totally agree with this post!",
        "Great job on explaining the topic.",
        "This is such a well-written article, thanks for sharing!",
        "Your insights are very helpful, I appreciate the effort.",
        "This is an interesting perspective, and I learned something new.",
        "I'm really impressed by the clarity of your explanation.",
        "I couldn't have said it better myself. Well done!",
        "This was a very informative read, thanks for posting it!",
        "I really appreciate the thoughtfulness of your comments.",
        "Such a positive contribution to the discussion!",
        
        # Hateful comments
        "You're such an idiot for believing this garbage!",
        "This is disgusting! People like you make me sick!",
        "I can't stand how stupid you are, do us all a favor and stop talking.",
        "Only morons would agree with this nonsense, you must be brain-dead!",
        "You're a worthless piece of trash, no one cares about your opinion!",
        "What a waste of time, you're nothing but a pathetic loser!",
        "It's hard to believe anyone is as dumb as you. Just shut up already.",
        "You're the reason why the world is so messed up. Go crawl under a rock!",
        "If ignorance was a crime, you'd be serving life. This is just sad.",
        "Do everyone a favor and disappear, your existence is pointless!"
    ]
}

mock_comments_df = pd.DataFrame(mock_data_comments)

# Hate Speech Prediction

## Functions

In [6]:
def preprocess_text(text):
    if isinstance(text, list):
        text = " ".join(text)
    elif not isinstance(text, str):
        print(f"Invalid text input: {text}")
        return ""
    
    # Lowercase the text
    text = text.lower()
    
    # Remove usernames, links, hashtags, and punctuation
    text = re.sub(r"@\w+", "", text)  # Remove usernames
    text = re.sub(r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE)  # Remove links
    text = re.sub(r"#(\w+)", r"\1", text)  # Remove hashtags
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    
    # Correct spelling
    # text = str(TextBlob(text).correct())
    
    # Remove any extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Preprocess comments dataset
def preprocess_comments(data):
    # Remove rows with missing text or duplicates
    data.dropna(subset=['text'], inplace=True)
    data = data.drop_duplicates(subset='text')
    data['preprocessed_text'] = [preprocess_text(text) for text in tqdm(data['text'], desc="Processing Texts")]
    
    return data

def tokenize_function(data):
    return tokenizer(data['text'], padding="max_length", truncation=True, max_length=128)

def predict_hate(dataset, model, tokenizer):
    # Create a DataLoader with batch size 16 and padding using the tokenizer
    loader = DataLoader(
        dataset, 
        batch_size=16, 
        collate_fn=DataCollatorWithPadding(tokenizer=tokenizer)
    )
    
    # Prepare lists to store texts and predictions
    texts, probabilities = [], []
    
    # Set the model to evaluation mode
    model.eval()
    
    with torch.no_grad():
        # Iterate over batches
        for batch in loader:
            # Get model outputs
            outputs = model(batch['input_ids'], attention_mask=batch['attention_mask'])
            # Convert logits to probabilities using softmax
            probs = torch.softmax(outputs.logits, dim=1)
            
            # Store text and probabilities
            for i in range(batch['input_ids'].size(0)):
                texts.append(tokenizer.decode(batch['input_ids'][i], skip_special_tokens=True))
                probabilities.append(probs[i].tolist())

    # Return results as a DataFrame
    return pd.DataFrame({
        'text': texts,
        'non_hate_probability': [p[0] for p in probabilities],  # Probability for non-hate speech
        'hate_probability': [p[1] for p in probabilities]   # Probability for hate speech
    })





# Function to create and train a model with given hyperparameters
def train_model_with_params(learning_rate, batch_size, num_epochs):
    training_args = TrainingArguments(
        output_dir="./results",
        evaluation_strategy="steps",
        eval_steps=500,
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=100,
        save_total_limit=1,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_validation,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )

    # Train the model
    trainer.train()
    
    # Evaluate the model on the validation set
    eval_results = trainer.evaluate()
    eval_loss = eval_results["eval_loss"]
    
    return eval_loss

## Training Model with labeled data from literature

In [7]:
mock_data_hate_labeled = preprocess_comments(mock_data_hate_labeled)
mock_dataset_hate_labeled = Dataset.from_pandas(mock_data_hate_labeled)

# Initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("GroNLP/hateBERT")
model = AutoModelForSequenceClassification.from_pretrained("GroNLP/hateBERT", num_labels=2)


# Split the dataset into train/test/validation with StratifiedShuffleSplit
splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.4, random_state=42)
train_indices, temp_indices = next(splitter.split(mock_dataset_hate_labeled['label'], mock_dataset_hate_labeled['label']))
train_set = mock_dataset_hate_labeled.select(train_indices)
temp_set = mock_dataset_hate_labeled.select(temp_indices)

# Further split temp into validation and test sets
temp_labels = temp_set['label']
splitter_temp = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=42)
validation_indices, test_indices = next(splitter_temp.split(temp_set['label'], temp_labels))
validation_set = temp_set.select(validation_indices)
test_set = temp_set.select(test_indices)


# Tokenize the datasets
tokenized_train = train_set.map(tokenize_function, batched=True)
tokenized_validation = validation_set.map(tokenize_function, batched=True)
tokenized_test = test_set.map(tokenize_function, batched=True)


Processing Texts: 100%|██████████| 20/20 [00:00<00:00, 50291.41it/s]
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at GroNLP/hateBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/12 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

In [8]:


# Define hyperparameter grid as desired
hyperparameter_grid = {
    'learning_rate': [1e-5, 3e-5, 5e-5],
    'per_device_train_batch_size': [8, 16],
    'num_train_epochs': [3, 5]
}

# Grid search
best_params = None
best_loss = np.inf

for lr in hyperparameter_grid['learning_rate']:
    for batch_size in hyperparameter_grid['per_device_train_batch_size']:
        for num_epochs in hyperparameter_grid['num_train_epochs']:
            print(f"Training with lr={lr}, batch_size={batch_size}, num_epochs={num_epochs}")
            loss = train_model_with_params(lr, batch_size, num_epochs)
            print(f"Validation Evaluation Loss: {loss}")
            
            if loss < best_loss:
                best_loss = loss
                best_params = (lr, batch_size, num_epochs)

print(f"Best Hyperparameters: learning_rate={best_params[0]}, batch_size={best_params[1]}, num_epochs={best_params[2]}")

# Train final model with the best hyperparameters on the full training data
best_learning_rate, best_batch_size, best_num_epochs = best_params


Training with lr=1e-05, batch_size=8, num_epochs=3


Step,Training Loss,Validation Loss


Validation Evaluation Loss: 0.6973150968551636
Training with lr=1e-05, batch_size=8, num_epochs=5


Step,Training Loss,Validation Loss


Validation Evaluation Loss: 0.6298425197601318
Training with lr=1e-05, batch_size=16, num_epochs=3


...

Validation Evaluation Loss: 0.004212190397083759
Training with lr=5e-05, batch_size=16, num_epochs=3


Step,Training Loss,Validation Loss


Validation Evaluation Loss: 0.0015345459105446935
Training with lr=5e-05, batch_size=16, num_epochs=5


Step,Training Loss,Validation Loss


Validation Evaluation Loss: 0.0006386138265952468
Best Hyperparameters: learning_rate=5e-05, batch_size=16, num_epochs=5


In [9]:
final_training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    eval_steps=500,
    learning_rate=best_learning_rate,
    per_device_train_batch_size=best_batch_size,
    per_device_eval_batch_size=best_batch_size,
    num_train_epochs=best_num_epochs,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False
)

final_trainer = Trainer(
    model=model,
    args=final_training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_validation,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# Train the model
final_trainer.train()

# Save the model
model.save_pretrained("./SupervisedTrainingHateBERT")
tokenizer.save_pretrained("./SupervisedTrainingHateBERT")

# Evaluate on the test set
test_results = final_trainer.evaluate(eval_dataset=tokenized_test)
test_loss = test_results["eval_loss"]

print(f"Test Set Evaluation Loss: {test_loss}")


Step,Training Loss,Validation Loss


Test Set Evaluation Loss: 0.0008615987026132643


## Predicting TikTok messages

In [10]:
# Load tokenizer and model for hate speech detection
tokenizer_hate = AutoTokenizer.from_pretrained("./SupervisedTrainingHateBERT")
model_hate = AutoModelForSequenceClassification.from_pretrained("./SupervisedTrainingHateBERT")

# Preprocess comments and get the preprocessed text
comments_data = preprocess_comments(mock_comments_df)
preprocessed_texts = comments_data['preprocessed_text'].to_list()

# Tokenize preprocessed texts using the hate speech tokenizer
encodings_hate = tokenizer_hate(preprocessed_texts, truncation=True, padding=True, max_length=128, return_tensors="pt")

# Create dataset for hate speech predictions
dataset_hate = Dataset.from_dict({
    'input_ids': encodings_hate['input_ids'],
    'attention_mask': encodings_hate['attention_mask']
})

# Make predictions
predictions_df = predict_hate(dataset_hate, model_hate, tokenizer_hate)

# You can then save the DataFrame to a CSV file if needed
predictions_file = "hateFirstTraining.csv"
predictions_df.to_csv(predictions_file, index=False)


Processing Texts: 100%|██████████| 20/20 [00:00<00:00, 50081.24it/s]


## Obtain pseudolabels and retraining HS prediction model with labels and pseudo-labels

In [11]:
# Define pseudolabels
hate_comments_predictions = pd.read_csv("hateFirstTraining.csv")

# Variables to control the pseudolabeling process
hate_probability_threshold_pseudolabel = False
non_hate_probability_threshold_pseudolabel = False

# Hate pseudolabeling based on probability threshold or quantile
if hate_probability_threshold_pseudolabel:
    probability_threshold = 0.95
    # Only keep rows with hate probability greater than the threshold
    hate_pseudolabels = hate_comments_predictions[hate_comments_predictions["hate_probability"] > probability_threshold]
    hate_pseudolabels["label"] = 1
else:
    quantile = 0.05
    # Get the top 5% of hate comments based on the hate probability
    top_5_percent_cutoff = hate_comments_predictions["hate_probability"].quantile(1 - quantile)
    hate_pseudolabels = hate_comments_predictions[hate_comments_predictions["hate_probability"] >= top_5_percent_cutoff]
    hate_pseudolabels["label"] = 1

# Non-hate pseudolabeling based on probability threshold or quantile
if non_hate_probability_threshold_pseudolabel:
    probability_threshold = 0.05
    # Only keep rows with hate probability less than the threshold
    non_hate_pseudolabels = hate_comments_predictions[hate_comments_predictions["hate_probability"] < probability_threshold]
    non_hate_pseudolabels["label"] = 0
else:
    quantile = 0.05
    # Get the bottom 5% of comments based on the hate probability
    bottom_5_percent_cutoff = hate_comments_predictions["hate_probability"].quantile(quantile)
    non_hate_pseudolabels = hate_comments_predictions[hate_comments_predictions["hate_probability"] <= bottom_5_percent_cutoff]
    non_hate_pseudolabels["label"] = 0

# Combine the hate and non-hate pseudolabels
hate_pseudolabels = pd.concat([hate_pseudolabels, non_hate_pseudolabels], ignore_index=True)

# Display the final pseudolabels dataset
hate_pseudolabels = hate_pseudolabels[["text", "label"]]
hate_pseudolabels

,text,label
0,youre a worthless piece of trash no one cares ...,1
1,such a positive contribution to the discussion,0


In [12]:
tokenizer = AutoTokenizer.from_pretrained("GroNLP/hateBERT")
model = AutoModelForSequenceClassification.from_pretrained("GroNLP/hateBERT", num_labels=2)

# Merge the DataFrames
hate_dataset = pd.concat([mock_data_hate_labeled, hate_pseudolabels])

hate_dataset = Dataset.from_pandas(hate_dataset)

# Split the dataset into train/test/validation with StratifiedShuffleSplit
splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_indices, temp_indices = next(splitter.split(hate_dataset['label'], hate_dataset['label']))
train_set = hate_dataset.select(train_indices)
temp_set = hate_dataset.select(temp_indices)

# Further split temp into validation and test sets
temp_labels = temp_set['label']
splitter_temp = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=42)
validation_indices, test_indices = next(splitter_temp.split(temp_set['label'], temp_labels))
validation_set = temp_set.select(validation_indices)
test_set = temp_set.select(test_indices)



# Tokenize the datasets
tokenized_train = train_set.map(tokenize_function, batched=True)
tokenized_validation = validation_set.map(tokenize_function, batched=True)
tokenized_test = test_set.map(tokenize_function, batched=True)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at GroNLP/hateBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/17 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [13]:
# Define hyperparameter grid
hyperparameter_grid = {
    'learning_rate': [1e-5, 3e-5, 5e-5],
    'per_device_train_batch_size': [8],
    'num_train_epochs': [5]
}

# Grid search
best_params = None
best_loss = np.inf

for lr in hyperparameter_grid['learning_rate']:
    for batch_size in hyperparameter_grid['per_device_train_batch_size']:
        for num_epochs in hyperparameter_grid['num_train_epochs']:
            print(f"Training with lr={lr}, batch_size={batch_size}, num_epochs={num_epochs}")
            loss = train_model_with_params(lr, batch_size, num_epochs)
            print(f"Evaluation Loss: {loss}")
            
            if loss < best_loss:
                best_loss = loss
                best_params = (lr, batch_size, num_epochs)

print(f"Best Parameters: learning_rate={best_params[0]}, batch_size={best_params[1]}, num_epochs={best_params[2]}")

# Use the best hyperparameters to train and save the final model
best_lr, best_batch_size, best_num_epochs = best_params


Training with lr=1e-05, batch_size=8, num_epochs=5


Step,Training Loss,Validation Loss


Evaluation Loss: 0.524088442325592
Training with lr=3e-05, batch_size=8, num_epochs=5


Step,Training Loss,Validation Loss


Evaluation Loss: 0.1373647302389145
Training with lr=5e-05, batch_size=8, num_epochs=5


Step,Training Loss,Validation Loss


Evaluation Loss: 0.017631273716688156
Best Parameters: learning_rate=5e-05, batch_size=8, num_epochs=5


In [14]:
final_training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    eval_steps=500,
    learning_rate=best_lr,
    per_device_train_batch_size=best_batch_size,
    per_device_eval_batch_size=best_batch_size,
    num_train_epochs=best_num_epochs,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False
)

final_trainer = Trainer(
    model=model,
    args=final_training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_validation,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# Train the model
final_trainer.train()

# Evaluate on the test set
test_results = final_trainer.evaluate(tokenized_test)
print("Test dataset results:", test_results)

# Save the final model and tokenizer
model.save_pretrained("./Semi_SupervisedTrainingHateBERT")
tokenizer.save_pretrained("./Semi_SupervisedTrainingHateBERT")

Step,Training Loss,Validation Loss


Test dataset results: {'eval_loss': 0.0033087944611907005, 'eval_runtime': 0.2363, 'eval_samples_per_second': 12.695, 'eval_steps_per_second': 4.232, 'epoch': 5.0}


('./Semi_SupervisedTrainingHateBERT/tokenizer_config.json',
 './Semi_SupervisedTrainingHateBERT/special_tokens_map.json',
 './Semi_SupervisedTrainingHateBERT/vocab.txt',
 './Semi_SupervisedTrainingHateBERT/added_tokens.json',
 './Semi_SupervisedTrainingHateBERT/tokenizer.json')

## Predicting final HS probabilities

In [15]:
# Load resources
tokenizer_hate = AutoTokenizer.from_pretrained("./Semi_SupervisedTrainingHateBERT")
model_hate = AutoModelForSequenceClassification.from_pretrained("./Semi_SupervisedTrainingHateBERT")

# Preprocess comments and get the preprocessed text
comments_data = preprocess_comments(mock_comments_df)
preprocessed_texts = comments_data['preprocessed_text'].to_list()

# Tokenize preprocessed texts using the hate speech tokenizer
encodings_hate = tokenizer_hate(preprocessed_texts, truncation=True, padding=True, max_length=128, return_tensors="pt")

# Create dataset for hate speech predictions
dataset_hate = Dataset.from_dict({
    'input_ids': encodings_hate['input_ids'],
    'attention_mask': encodings_hate['attention_mask']
})

# Make predictions
predictions_df = predict_hate(dataset_hate, model_hate, tokenizer_hate)

# You can then save the DataFrame to a CSV file if needed
predictions_file = "finalHatePredictions.csv"
predictions_df.to_csv(predictions_file, index=False)
predictions_df

Processing Texts: 100%|██████████| 20/20 [00:00<00:00, 58991.62it/s]


,text,non_hate_probability,hate_probability
0,i totally agree with this post,0.998085,0.001915
1,great job on explaining the topic,0.998515,0.001485
2,this is such a wellwritten article thanks for ...,0.998351,0.001649
3,your insights are very helpful i appreciate th...,0.998318,0.001682
4,this is an interesting perspective and i learn...,0.997467,0.002533
5,im really impressed by the clarity of your exp...,0.997870,0.002130
6,i couldnt have said it better myself well done,0.995290,0.004710
7,this was a very informative read thanks for po...,0.998488,0.001512
8,i really appreciate the thoughtfulness of your...,0.998372,0.001628
9,such a positive contribution to the discussion,0.998791,0.001209


# Counter Speech Prediction

## Functions

In [16]:
# Define the function to prepare data
def prepare_data(subdialogues_df):
    pairs = []
    labels = []

    for i in range(0, len(subdialogues_df) - 1, 2):
        text1 = preprocess_text(subdialogues_df.iloc[i]['text'])
        text2 = preprocess_text(subdialogues_df.iloc[i + 1]['text'])
        combined_text = "[CLS] " + text1 + " [SEP] " + text2
        combined_label = [subdialogues_df.iloc[i]['label'], subdialogues_df.iloc[i + 1]['label']]
        pairs.append(combined_text)
        labels.append(combined_label)

    return pd.DataFrame({'text': pairs, 'label': labels})

# Define the function to prepare data
def prepare_data_TikTok(subdialogues_df):
    pairs = []
    
    for i in range(0, len(subdialogues_df) - 1, 2):
        text1 = preprocess_text(subdialogues_df.iloc[i]['text'])
        text2 = preprocess_text(subdialogues_df.iloc[i + 1]['text'])
        combined_text = "[CLS] " + text1 + " [SEP] " + text2
        pairs.append(combined_text)

    return pd.DataFrame({'text': pairs})


# Define the function to prepare data
def prepare_data_TikTok(df):
    pairs = []
    grouped = df.groupby('dialogue_id')
    for _, group in grouped:
        if len(group) % 2 != 0:
            group = group[:-1]
        
        for i in range(0, len(group), 2):
            combined_text = "[CLS] " + preprocess_text(group.iloc[i]['text']) + " [SEP] " + preprocess_text(group.iloc[i+1]['text'])
            pairs.append(combined_text)
    return pd.DataFrame({'text': pairs})


def predict(dataloader, model):
    model.eval()
    all_preds = []
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            probs = torch.nn.functional.softmax(logits, dim=1).cpu().numpy()
            
            all_preds.extend(probs)
    return all_preds


def assign_labels(df, threshold=0.8, quantile=None, is_counter_speech=True):
    labels = []
    for _, row in df.iterrows():
        prob = row[['probability_1', 'probability_2']].values
        max_prob = max(prob)
        index_max_prob = prob.argmax()
        
        if prob[1] > threshold:
            label = [0,1]
        elif prob[0] > threshold:
            label = [1,0]
        else:
            label = [0,0]
        
        labels.append(label)
    return labels

In [82]:
# Dataset class
class DialogueDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts.iloc[idx]
        label = self.labels.iloc[idx]
        inputs = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=False,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'labels': torch.tensor(label.values, dtype=torch.float)
        }

class PredictionDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=128):
        self.texts = df['text'].tolist()
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        inputs = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten()
        }

## Training Model with labeled data from literature

In [83]:
data = {
    'dialogue_id': [
        '1001', '1001', '1002', '1002', '1003',
        '1003', '1004', '1004', '1005', '1005',
        '1006', '1006', '1007', '1007', '1008',
        '1008', '1009', '1009', '1010', '1010',
        '1011', '1011', '1012', '1012', '1013',
        '1013', '1014', '1014', '1015', '1015',
        '1016', '1016', '1017', '1017', '1018',
        '1018', '1019', '1019', '1020', '1020'
    ],
    'text': [
        "You're so stupid, I can't believe anyone listens to you.",  # HS
        "It's important to stay respectful even when we disagree.",  # CN
        "Why do people like you always spread misinformation?",  # HS
        "Everyone has different opinions, and that's okay.",  # CN
        "People like you are ruining the discussion with your hatred.",  # HS
        "Let's focus on constructive dialogue rather than insults.",  # CN
        "I find your comments deeply offensive and ignorant.",  # HS
        "We should be more empathetic and try to understand each other's perspectives.",  # CN
        "This kind of hate speech should not be tolerated.",  # HS
        "Respectful conversation is key to resolving conflicts.",  # CN
        "Your views are so outdated and discriminatory.",  # HS
        "We can have a meaningful debate without resorting to personal attacks.",  # CN
        "You only spread negativity and hate.",  # HS
        "Constructive criticism is a way to improve, not to insult.",  # CN
        "This rhetoric only fosters division and hatred.",  # HS
        "Let's engage in discussions that promote mutual respect.",  # CN
        "Hate speech has no place in our society.",  # HS
        "Understanding and compassion can bridge gaps between us.",  # CN
        "Your comment is a perfect example of toxic behavior.",  # HS
        "We should strive for a more inclusive and respectful dialogue.",  # CN
        "I can't believe how ignorant some people can be.",  # HS
        "Engaging in respectful dialogue can help us learn from each other.",  # CN
        "Your argument is completely unfounded and offensive.",  # HS
        "Let's focus on finding common ground rather than attacking each other.",  # CN
        "Your language is very hurtful and damaging.",  # HS
        "Promoting understanding is essential for constructive discussions.",  # CN
        "This kind of dialogue only escalates conflict.",  # HS
        "We should aim for conversations that are both respectful and productive.",  # CN
        "Your statements are harmful and unnecessary.",  # HS
        "Open-mindedness and respect are crucial for meaningful exchanges.",  # CN
        "This kind of negativity is detrimental to productive conversation.",  # HS
        "Constructive dialogue involves listening and understanding.",  # CN
        "Your comments only serve to create further division.",  # HS
        "Let's work towards resolving our differences with mutual respect.",  # CN
        "This type of rhetoric is unacceptable in any discussion.",  # HS
        "We should always strive to communicate with empathy and respect.",  # CN
        "Your behavior is completely unacceptable and damaging.",  # HS
        "Respectful discourse is the foundation of any healthy discussion.",  # CN
        "I do not like you.", # HS
        "You should not answer like this." # CN
    ],
    'turn_id': [
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1
    ],
    'label': [
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1
    ]
}

mock_dialogues_df = pd.DataFrame(data)
mock_dialogues_df.head()

,dialogue_id,text,turn_id,label
0,1001,"You're so stupid, I can't believe anyone liste...",0,0
1,1001,It's important to stay respectful even when we...,1,1
2,1002,Why do people like you always spread misinform...,0,0
3,1002,"Everyone has different opinions, and that's okay.",1,1
4,1003,People like you are ruining the discussion wit...,0,0


In [80]:
pairs_df = prepare_data(mock_dialogues_df)
pairs_df.head()

,text,label
0,[CLS] youre so stupid i cant believe anyone li...,"[0, 1]"
1,[CLS] why do people like you always spread mis...,"[0, 1]"
2,[CLS] people like you are ruining the discussi...,"[0, 1]"
3,[CLS] i find your comments deeply offensive an...,"[0, 1]"
4,[CLS] this kind of hate speech should not be t...,"[0, 1]"


In [81]:
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

X = pairs_df['text']
y = pd.DataFrame(pairs_df['label'].tolist(), columns=['label_1', 'label_2'])

# Split into train (60%), val (20%), test (20%)
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)



# Tokenizer and datasets
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_dataset = DialogueDataset(X_train, y_train, tokenizer, max_len=128)
val_dataset = DialogueDataset(X_val, y_val, tokenizer, max_len=128)
test_dataset = DialogueDataset(X_test, y_test, tokenizer, max_len=128)

# Load model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Training args
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy='epoch',
    save_strategy='epoch',
    report_to="none",
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Train
trainer.train()

# Save model
os.makedirs("./SupervisedTrainingDialogueBERT", exist_ok=True)
model.save_pretrained("./SupervisedTrainingDialogueBERT")
tokenizer.save_pretrained("./SupervisedTrainingDialogueBERT")

# Evaluate on the test dataset
test_results = trainer.evaluate(test_dataset)
print("Test dataset results:", test_results)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tensor([0., 1.])
tensor([0., 1.])
tensor([0., 1.])
tensor([0., 1.])
tensor([0., 1.])
tensor([0., 1.])
tensor([0., 1.])
tensor([0., 1.])
tensor([0., 1.])
tensor([0., 1.])
tensor([0., 1.])
tensor([0., 1.])


Epoch,Training Loss,Validation Loss
1,No log,0.739102
2,No log,0.736809
3,No log,0.732828


tensor([0., 1.])
tensor([0., 1.])
tensor([0., 1.])
tensor([0., 1.])
tensor([0., 1.])
tensor([0., 1.])
tensor([0., 1.])
tensor([0., 1.])
tensor([0., 1.])
tensor([0., 1.])
tensor([0., 1.])
tensor([0., 1.])
tensor([0., 1.])
tensor([0., 1.])
tensor([0., 1.])
tensor([0., 1.])
tensor([0., 1.])
tensor([0., 1.])
tensor([0., 1.])
tensor([0., 1.])
tensor([0., 1.])
tensor([0., 1.])
tensor([0., 1.])
tensor([0., 1.])
tensor([0., 1.])
tensor([0., 1.])
tensor([0., 1.])
tensor([0., 1.])
tensor([0., 1.])
tensor([0., 1.])
tensor([0., 1.])
tensor([0., 1.])
tensor([0., 1.])
tensor([0., 1.])
tensor([0., 1.])
tensor([0., 1.])
tensor([0., 1.])
tensor([0., 1.])
tensor([0., 1.])
tensor([0., 1.])


Test dataset results: {'eval_loss': 0.7075212001800537, 'eval_runtime': 0.2827, 'eval_samples_per_second': 14.15, 'eval_steps_per_second': 3.538, 'epoch': 3.0}


## Predicting TikTok Dialogues

In [73]:

data = {
    'dialogue_id': [
        '3001', '3001', '3002', '3002', '3003',
        '3003', '3004', '3004', '3005', '3005',
        '3006', '3006', '3007', '3007', '3008',
        '3008', '3009', '3009', '3010', '3010',
        '3011', '3011', '3012', '3012', '3013',
        '3013', '3014', '3014', '3015', '3015',
        '3016', '3016', '3017', '3017', '3018',
        '3018', '3019', '3019', '3020', '3020'
    ],
    'text': [
        "People like you are ruining this community with your hate.",  # HS
        "Everyone deserves respect, regardless of their opinions.",  # CN
        "Your arguments are so backward and offensive.",  # HS
        "It's important to engage in discussions with empathy and understanding.",  # CN
        "Why do people like you always spread negativity?",  # HS
        "Constructive dialogue can bridge gaps and resolve conflicts.",  # CN
        "Your comment is harmful and discriminatory.",  # HS
        "Let's focus on positive and respectful communication.",  # CN
        "You should be ashamed of your prejudiced views.",  # HS
        "Everyone has different perspectives and that's okay.",  # CN
        "This kind of rhetoric only perpetuates hatred.",  # HS
        "Promoting mutual respect and understanding is crucial.",  # CN
        "Your behavior is unacceptable and damaging.",  # HS
        "We should strive for more inclusive and respectful conversations.",  # CN
        "Hate speech has no place in a healthy dialogue.",  # HS
        "Constructive criticism helps us grow, not tear us down.",  # CN
        "This kind of negativity only fosters division.",  # HS
        "Let's engage in discussions that promote empathy and learning.",  # CN
        "Your language is hurtful and unnecessary.",  # HS
        "Respectful dialogue is essential for productive conversations.",  # CN
        "People like you are spreading false information.",  # HS
        "We can have disagreements without resorting to personal attacks.",  # CN
        "Your comments are divisive and harmful.",  # HS
        "Understanding different viewpoints can lead to meaningful discussions.",  # CN
        "You only contribute to the problem with your hateful speech.",  # HS
        "Empathy and respect are the foundation of any healthy debate.",  # CN
        "This kind of discourse is unacceptable and damaging.",  # HS
        "Promoting kindness and open-mindedness can make a difference.",  # CN
        "Your views are toxic and unproductive.",  # HS
        "Let's work towards creating a more respectful and understanding environment.",  # CN
        "Your comments are inflammatory and disrespectful.",  # HS
        "Focusing on common ground can help us resolve conflicts.",  # CN
        "Your attitude is completely unacceptable.",  # HS
        "We should always strive for constructive and respectful conversations.",  # CN
        "Your statements are prejudiced and harmful.",  # HS
        "Encouraging respectful dialogue can lead to positive change.",  # CN
        "Your argument is completely unfounded and offensive.",  # HS
        "Let's focus on finding common ground rather than attacking each other.",  # CN
        "Your language is very hurtful and damaging.",  # HS
        "Promoting understanding is essential for constructive discussions.",  # CN
    ],
    'turn_id': [
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1
    ]
}

mock_dialogues_TikTok_df = pd.DataFrame(data)

In [74]:
# Load the model and tokenizer
tokenizer = BertTokenizer.from_pretrained("./SupervisedTrainingDialogueBERT")
model = BertForSequenceClassification.from_pretrained("./SupervisedTrainingDialogueBERT")

pairs_df_TikTok = prepare_data_TikTok(mock_dialogues_TikTok_df)
# Create the dataset
prediction_dataset = PredictionDataset(pairs_df_TikTok, tokenizer)

# Create a dataloader
dataloader = DataLoader(prediction_dataset, batch_size=8)

# Get predictions
probs = predict(dataloader, model)

pairs_df_TikTok["probabilities"] = probs
print("Probabilities:\n", probs)
print(len(probs))


Probabilities:
 [array([0.46400982, 0.5359902 ], dtype=float32), array([0.45302135, 0.5469786 ], dtype=float32), array([0.50846034, 0.49153963], dtype=float32), array([0.5130343 , 0.48696575], dtype=float32), array([0.502367  , 0.49763295], dtype=float32), array([0.4674845, 0.5325154], dtype=float32), array([0.50496083, 0.4950392 ], dtype=float32), array([0.47799498, 0.522005  ], dtype=float32), array([0.4412687 , 0.55873126], dtype=float32), array([0.48659128, 0.5134087 ], dtype=float32), array([0.43824032, 0.5617597 ], dtype=float32), array([0.5100657 , 0.48993438], dtype=float32), array([0.462044, 0.537956], dtype=float32), array([0.47992748, 0.52007246], dtype=float32), array([0.51272655, 0.48727348], dtype=float32), array([0.4572117 , 0.54278827], dtype=float32), array([0.4852293 , 0.51477075], dtype=float32), array([0.50630677, 0.4936932 ], dtype=float32), array([0.48597038, 0.5140296 ], dtype=float32), array([0.47975436, 0.5202456 ], dtype=float32)]
20


## Obtain pseudolabels and retraining CS prediction model with labels and pseudo-labels

In [75]:
pairs_df_TikTok[['probability_1', 'probability_2']] = pd.DataFrame(pairs_df_TikTok['probabilities'].tolist(), index=pairs_df_TikTok.index)

# Variables to control the pseudolabeling process
counter_speech_probability_threshold_pseudolabel = False
non_counter_speech_probability_threshold_pseudolabel = False

# --- Pseudolabeling for high-probability counter speech ---
if counter_speech_probability_threshold_pseudolabel:
    probability_threshold = 0.95
    top_df = pairs_df_TikTok[pairs_df_TikTok['probability_2'] > probability_threshold].copy()
    top_df['label'] = assign_labels(top_df, threshold=probability_threshold, is_counter_speech=True)
else:
    quantile = 0.05
    top_cutoff = pairs_df_TikTok['probability_2'].quantile(1 - quantile)
    top_df = pairs_df_TikTok[pairs_df_TikTok['probability_2'] >= top_cutoff].copy()
    top_df['label'] = assign_labels(top_df, quantile=quantile, is_counter_speech=True)

# --- Pseudolabeling for low-probability (non-counter) speech ---
if non_counter_speech_probability_threshold_pseudolabel:
    probability_threshold = 0.05
    bottom_df = pairs_df_TikTok[pairs_df_TikTok['probability_2'] < probability_threshold].copy()
    bottom_df['label'] = assign_labels(bottom_df, threshold=probability_threshold, is_counter_speech=False)
else:
    quantile = 0.05
    bottom_cutoff = pairs_df_TikTok['probability_2'].quantile(quantile)
    bottom_df = pairs_df_TikTok[pairs_df_TikTok['probability_2'] <= bottom_cutoff].copy()
    bottom_df['label'] = assign_labels(bottom_df, quantile=quantile, is_counter_speech=False)

# --- Combine top and bottom pseudolabels ---
counter_speech_pseudolabels = pd.concat([top_df, bottom_df])[["text", "label"]]
print(counter_speech_pseudolabels)

# --- Combine with existing labeled data ---
pairs_df_labeled = prepare_data(mock_dialogues_df)
pairs_df = pd.concat([pairs_df_labeled, counter_speech_pseudolabels]).reset_index(drop=True)
pairs_df

                                                 text   label
10  [CLS] people like you are spreading false info...  [0, 0]
3   [CLS] your comment is harmful and discriminato...  [0, 0]


,text,label
0,[CLS] youre so stupid i cant believe anyone li...,"[0, 1]"
1,[CLS] why do people like you always spread mis...,"[0, 1]"
2,[CLS] people like you are ruining the discussi...,"[0, 1]"
3,[CLS] i find your comments deeply offensive an...,"[0, 1]"
4,[CLS] this kind of hate speech should not be t...,"[0, 1]"
5,[CLS] your views are so outdated and discrimin...,"[0, 1]"
6,[CLS] you only spread negativity and hate [SEP...,"[0, 1]"
7,[CLS] this rhetoric only fosters division and ...,"[0, 1]"
8,[CLS] hate speech has no place in our society ...,"[0, 1]"
9,[CLS] your comment is a perfect example of tox...,"[0, 1]"


In [76]:
X = pairs_df['text']
y = pd.DataFrame(pairs_df['label'].tolist(), columns=['label_1', 'label_2'])


# Split into train (60%), val (20%), test (20%)
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_validation, y_train, y_validation = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)

# Tokenizer and datasets
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_dataset = DialogueDataset(X_train, y_train, tokenizer, max_len=128)
val_dataset = DialogueDataset(X_validation, y_validation, tokenizer, max_len=128)
test_dataset = DialogueDataset(X_test, y_test, tokenizer, max_len=128)


# Load model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy='epoch',  # Evaluate on val set each epoch
    save_strategy='epoch',
    report_to="none",
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Train the model
trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,No log,0.728548
2,No log,0.727573
3,No log,0.725780


TrainOutput(global_step=6, training_loss=0.6690843105316162, metrics={'train_runtime': 12.9471, 'train_samples_per_second': 2.781, 'train_steps_per_second': 0.463, 'total_flos': 2367999498240.0, 'train_loss': 0.6690843105316162, 'epoch': 3.0})

In [77]:
os.makedirs("./SemiSupervisedTrainingDialogueBERT", exist_ok=True)
model.save_pretrained("./SemiSupervisedTrainingDialogueBERT")
tokenizer.save_pretrained("./SemiSupervisedTrainingDialogueBERT")

# Evaluate on the test dataset
test_results = trainer.evaluate(test_dataset)
print("Test dataset results:", test_results)

Test dataset results: {'eval_loss': 0.6908215284347534, 'eval_runtime': 0.327, 'eval_samples_per_second': 15.29, 'eval_steps_per_second': 3.058, 'epoch': 3.0}


## Predicting final CS probabilities

In [69]:

# Load the model and tokenizer
tokenizer = BertTokenizer.from_pretrained("./SemiSupervisedTrainingDialogueBERT")
model = BertForSequenceClassification.from_pretrained("./SemiSupervisedTrainingDialogueBERT")

pairs_df_TikTok = prepare_data_TikTok(mock_dialogues_TikTok_df)
# Create the dataset
prediction_dataset = PredictionDataset(pairs_df_TikTok, tokenizer)

# Create a dataloader
dataloader = DataLoader(prediction_dataset, batch_size=8)

# Get predictions
probs = predict(dataloader, model)

pairs_df_TikTok["probabilities"] = probs
print("Probabilities:\n", probs)
print(len(probs))


Probabilities:
 [array([0.46406472, 0.5359353 ], dtype=float32), array([0.4526586, 0.5473414], dtype=float32), array([0.5084366 , 0.49156335], dtype=float32), array([0.5125879 , 0.48741212], dtype=float32), array([0.5024599 , 0.49754018], dtype=float32), array([0.46737164, 0.53262836], dtype=float32), array([0.5049363 , 0.49506372], dtype=float32), array([0.4778811, 0.5221189], dtype=float32), array([0.44102094, 0.55897903], dtype=float32), array([0.48624855, 0.51375145], dtype=float32), array([0.43817946, 0.56182057], dtype=float32), array([0.5095751 , 0.49042487], dtype=float32), array([0.46177673, 0.53822327], dtype=float32), array([0.47973704, 0.52026296], dtype=float32), array([0.51225865, 0.48774135], dtype=float32), array([0.45677826, 0.5432217 ], dtype=float32), array([0.48526284, 0.5147372 ], dtype=float32), array([0.50586224, 0.49413773], dtype=float32), array([0.48558873, 0.5144112 ], dtype=float32), array([0.47942236, 0.52057767], dtype=float32)]
20
